# Exploring Hacker News
In this project we will explore whether `Ask HN` and `Show HN` receive more comments on average, as well as answer other questions.

## Open the dataset

In [2]:
from csv import reader

opened_file = open('datasets/hacker_news.csv')
read_file = reader(opened_file)
hn = list(read_file)

# Extract the first row, which contains labels
headers = hn[0]
hn = hn[1:]

print(headers)
print(hn[:5])

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']
[['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01'], ['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']]


## Separate Ask, Show and other posts

In [3]:
ask_posts = []
show_posts = []
other_posts = []

for row in hn:
    title = row[1].lower()
    if title.startswith('ask hn'):
        ask_posts.append(row)
    elif title.startswith('show hn'):
        show_posts.append(row)
    else:
        other_posts.append(row)
        
print("Ask: {}".format(len(ask_posts)))
print("Show: {}".format(len(show_posts)))
print("Other: {}".format(len(other_posts)))

print(ask_posts[:5])
print(show_posts[:5])
print(other_posts[:5])


Ask: 1744
Show: 1162
Other: 17194
[['12296411', 'Ask HN: How to improve my personal website?', '', '2', '6', 'ahmedbaracat', '8/16/2016 9:55'], ['10610020', 'Ask HN: Am I the only one outraged by Twitter shutting down share counts?', '', '28', '29', 'tkfx', '11/22/2015 13:43'], ['11610310', 'Ask HN: Aby recent changes to CSS that broke mobile?', '', '1', '1', 'polskibus', '5/2/2016 10:14'], ['12210105', 'Ask HN: Looking for Employee #3 How do I do it?', '', '1', '3', 'sph130', '8/2/2016 14:20'], ['10394168', 'Ask HN: Someone offered to buy my browser extension from me. What now?', '', '28', '17', 'roykolak', '10/15/2015 16:38']]
[['10627194', 'Show HN: Wio Link  ESP8266 Based Web of Things Hardware Development Platform', 'https://iot.seeed.cc', '26', '22', 'kfihihc', '11/25/2015 14:03'], ['10646440', 'Show HN: Something pointless I made', 'http://dn.ht/picklecat/', '747', '102', 'dhotson', '11/29/2015 22:46'], ['11590768', 'Show HN: Shanhu.io, a programming playground powered by e8vm',

In [4]:
def average_comments(a_list, index):
    total_comments = 0

    for row in a_list:
        num_comments = int(row[index])
        total_comments += num_comments

    avg_comments = total_comments / len(a_list)    
    return avg_comments

avg_ask_comments = average_comments(ask_posts, 4)
avg_show_comments = average_comments(show_posts, 4)

print("Ask posts average comments: {:.2f}".format(avg_ask_comments))
print("Show posts average comments: {:.2f}".format(avg_show_comments))

Ask posts average comments: 14.04
Show posts average comments: 10.32


On average, ask posts receive almost 4 more comments than show posts.

## Determine whether Ask posts created at a certain time are more likely to receive comments

In [11]:
import datetime as dt

result_list = []

for row in ask_posts:
    entry = []
    entry.append(row[6])
    entry.append(int(row[4]))
    result_list.append(entry)
    
counts_by_hour = {}
comments_by_hour = {}

for result in result_list:
    created_date = dt.datetime.strptime(result[0], "%m/%d/%Y %H:%M")
    hour = created_date.strftime('%H')
    
    if hour not in counts_by_hour:
        counts_by_hour[hour] = 1
        comments_by_hour[hour] = result[1]
    else:
        counts_by_hour[hour] += 1
        comments_by_hour[hour] += result[1]
    

## Calculate the average comments per hour

In [12]:
avg_by_hour = []

for hour in comments_by_hour:
    count = counts_by_hour[hour]
    comments = comments_by_hour[hour]
    
    average = comments / count
    
    avg_by_hour.append([hour, average])    

[['09', 5.5777777777777775], ['13', 14.741176470588234], ['10', 13.440677966101696], ['14', 13.233644859813085], ['16', 16.796296296296298], ['23', 7.985294117647059], ['12', 9.41095890410959], ['17', 11.46], ['15', 38.5948275862069], ['21', 16.009174311926607], ['20', 21.525], ['02', 23.810344827586206], ['18', 13.20183486238532], ['03', 7.796296296296297], ['05', 10.08695652173913], ['19', 10.8], ['01', 11.383333333333333], ['22', 6.746478873239437], ['08', 10.25], ['04', 7.170212765957447], ['00', 8.127272727272727], ['06', 9.022727272727273], ['07', 7.852941176470588], ['11', 11.051724137931034]]


## Print top five averages

In [21]:
swap_avg_by_hour = []

for row in avg_by_hour:
    swap_avg_by_hour.append([row[1], row[0]])
    
sorted_swap = sorted(swap_avg_by_hour, reverse=True)

print("Top Five Hours for Ask Posts Comments")

for average, hour in sorted_swap[:5]:
    hour = dt.datetime.strptime(hour, "%H")
    formatted_hour = hour.strftime("%H:00")
    print("{}: {:.2f} average comments per post".format(formatted_hour, average))

Top Five Hours for Ask Posts Comments
15:00: 38.59 average comments per post
02:00: 23.81 average comments per post
20:00: 21.52 average comments per post
16:00: 16.80 average comments per post
21:00: 16.01 average comments per post


# Conclusion
To garner comments, make an Ask post at about 3pm Eastern Time.